참고링크
- https://velog.io/@dscwinterstudy/2020-02-02-0002-%EC%9E%91%EC%84%B1%EB%90%A8-pwk63r14ez#31-%EC%B6%94%EB%A1%A0-%EA%B8%B0%EB%B0%98-%EA%B8%B0%EB%B2%95%EA%B3%BC-%EC%8B%A0%EA%B2%BD%EB%A7%9D
- https://asthtls.tistory.com/1058

이번 장에서는 더 강력한 기법인 추론 기반 기법을 살펴본다. <br><br>
추론 기반 기법의 추론 과정에서 신경망을 이용한다. <br>
여기서 __word2vec__이 등장한다. <br>
이번 장에서는 __word2vec__의 구조를 차분히 들여다보고 구현해본다. <br><br>


이번 장의 목표는 단순한 __word2vec__구현하기이다. <br>
다음 장에서는 __word2vec__을 완성시킨다. 

## 3.1 추론 기반 기법과 신경망
단어를 벡터로 표현하는 방법은 크게 두 부분이 있다.
1. 통계 기반 기법
2. 추론 기반 기법

단어의 의미를 얻는 방식은 서로 크게 다르지만, 그 __배경에는 모두 분포 가설이 있다.__ <br><br>

이번 절에서는 통계 기반 기법의 문제를 지적하고, 그 대안인 초론 기반 기법의 이점을 거시적 관점에서 설명한다.

##### 3.1.1 통계 기반 기법의 문제점
지금까지 본 것처럼 통계 기반 기법에서는 주변 단어의 빈도를 기초로 단어로 표현했다.<br>
구체적으로는 단어의 동시발생 행렬을 만들고, 그 행렬에 SVD를 적용하여 밀집벡터:단어의 분산 표현을 얻었다. <br><br>&nbsp;
 __그러나 이 방식은 대규모 말뭉치를 다룰 때 문제가 발생한다.__ <br><br>
 
현업에서 다루는 말뭉치의 어휘 수는 어마어마하다. <br>
이런 거대 행렬에 SVD를 적용하는 일은 현실적이지 않다.
 
> SVD를 $n$ x $n$ 행렬에 적용하는 비용은 $O(n^3)$이다.

- 통계 기반 기법은 말뭉치 전체의 통계
- 동시 발생 행렬과 PPMI 등을 이용해 단 1회의 처리
- SVD등으로 단어의 분산 표현을 얻는다.

한편, 추론 기반 기법에서는 예컨대 신경망을 이용하는 경우는 미니배치로 학습하는 것이 일반적이다. <br><br>
미니배치 학습에서는 신경망이 한번에 소량, 미니배치의 학습 샘플씩 반복해서 학습하며 가중치를 갱신한다.

> ![](img/minibatch1.png)
> 통계 기반 기법은 학습데이터를 __한꺼번에 처리__한다.<br>
> 추론 기반 기법은 학습 데이터의 일부를 사용하여 순차적으로 학습한다. <br>
> 미니배치 학습, 말뭉치의 어휘 수가 많아 SVD 등 계산량이 큰 작업을 처리하기 어려운 경우에도 __신경망을 학습__시킬 수 있다는 의미이다. <br>
__데이터를 작게 나눠 학습하기 때문이다.__

##### 3.1.2 추론 기반 기법 개요

추론 기반 기법에서는 추론이 주된 작업이다. <br>
- 추론이란? <br>
 => 주변 단어와 맥락이 주어졌을 때 __?__에 무슨 단어가 들어가는지를 추측하는 작업이다.
 ![](img/infer1.png)
 
 추론 문제를 풀고 학습하는 것이 추론 기반 기법이 다루는 문제이다.
 이러한 추론 문제를 반복해서 풀면서 단어의 출현 패턴을 학습하는 것이다.
 
<br>

![](img/infer2.png)

__추론 기반 기법에는 어떠한 모델이 등장한다.__ <br>
> 우리는 이 모델로 신경망을 사용한다. <br>
> 모델은 맥락 정보를 입력받아 출현할 수 있는 각 단어의 출현 확률을 출력한다. <br>
> 이러한 틀 안에서 말뭉치를 사용해 모델이 올바른 추측을 내놓도록 학습시킨다. <br>
> 그리고 그 학습의 결과로 단어의 분산 표현을 얻는 것이 추론 기반 기법의 전체 그림이다.

__추론 기반 기법도 통계 기반 기법처럼 분포 가설에 기초한다.__ <br>
> 분포 가설이란, 단어의 의미는 주변 단어에 의해 형성된다는 가설로, 이를 추측 문제로 귀결시켰다. <br><br>
> 이처럼 두 기법 모두 분포 가설에 근거하는 단어의 동시발생 가능성을 얼마나 잘 모델링 하는가가 중요한 연구 주제이다.

##### 3.1.3 신경망에서의 단어 처리
지금부터 신경망을 이용해 단어를 처리해보자. <br>
__단어를 있는 그대로 처리할 수 없으니 고정 길이의 벡터로 변환해야한다.__ <br>
이때 사용하는 대표적인 방법이 단어를 One-hot 표현으로 변환하는 것이다. <br>
원핫 표현이란, 벡터의 원소 중 하나만 1이고, 나머지는 모두 0인 벡터를 말한다.<br>

> ![](img/onehot1.png) <br>
> 단어는 텍스트, 단어ID, 원핫 표현 형태로 나타낼 수 있다.

__◎ 단어를 원핫 표현으로 변환하는 방법__
- 먼저 총 어휘 수 만큼의 원소를 갖는 벡터를 준비
- index가 단어 ID와 같은 원소를 1로, 나머지는 모두 0으로 설정한다. <br>
 => 이처럼 단어를 고정 길이 벡터로 변환하면, 신경망의 입력층은 뉴런의 수를 고정할 수 있다. 
<br><br>
 

![](img/input1.png)
> 단어를 벡터로 나타낼 수 있고, 신경망을 구성하는 계층들은 벡터를 처리할 수 있다.<br>
> 다시 말해, 단어를 신경망으로 처리할 수 있다는 뜻이다.

![](img/input2.png)
> 화살표에는 가중치와 매개변수가 존재하며, 입력층 뉴런과의 가중합이 은닉층 뉴런이 된다. <br>
> 참고로 이번 장에서 사용하는 완전연결계층에서는 편향을 생략했다. <br>
> 편향을 이용하지 않은 완전연결계층은 단순한 행렬 곱 계산에 해당된다.

![](img/input3.png)
<br><br>
![](img/input4.png)

---
## 3.2 단순한 word2vec
앞 절에서 추론 기반 기법을 배우고, 신경망으로 단어를 처리하는 방법을 코드로 살펴보았다.<br>
이제 word2vec을 구현할 차례이다. <br><br>

지금부터 할 일은 모델을 신경망으로 구축하는 것이다. <br>
이번 절에서 사용할 신경망은 word2vec에서 제안하는 CBOW(Continuos-bag-of-words)모델이다.

##### 3.2.1 CBOW 모델의 추론 처리
CBOW 모델은 맥락으로부터 타깃을 추측하는 용도의 신경망이다. <br>
__타깃은 중앙 단어이고, 그 주변 단어들이 맥락이다.__ <br>
우리는 이 CBOW 모델이 가능한 한 정확하게 추론하도록 훈련시켜서 __단어의 분산 표현__을 얻어낼 것이다. <br><br>

CBOW 모델의 입력은 맥락이다. <br>
가장 먼저, 이 맥락을 One-hot 표현으로 변환하여 , CBOW 모델이 처리할 수 있도록 준비한다.
![](img/cbow1.png)
> - 그림에서 입력층이 2개인 이유 <br>
=> 맥락으로 고려한 단어를 2개로 정했기 때문
> - 은닉층의 뉴런은 입력층의 완전 연결계층에 의해 반환되는 값<br>
=> 입력층이 여러개면 전체를 평균
- 위 그림에서 출력층의 뉴런은 총 7개인데, 중요한 것은 이 뉴런 하나하나가 각각 단어에 대응한다는 점이다. 
- 그리고, 출력층 뉴런은 각 단어의 점수르 ㄹ뜻하며, 값이 높을수록 대응단어의 출현 확률도 높아진다.
- 여기서 점수란, 확률로 해석되기 전의 값이고, 이 점수에 softmax함수를 적용해서 확률을 얻을 수 있다.
- 점수를 Softmax 계층에 통과시킨 후의 뉴런을 출력층이라고 한다.

![](img/neuron1.png)

> - __은닉층의 뉴런수를 입력층의 뉴런 수보다 적게하는 것이 중요한 핵심이다.__ <br>
> - 이렇게 해야 은닉층에는 단어 예측에 필요한 정보를 간결하게 담게 되며, 결과적으로 밀집 벡터 표현을 얻을 수 있다.<br>
> - 이때 은닉층 정보는 인간이 히해할 수 없는 코드로 쓰여 있다.(인코딩) <br>
> - 한편, 은닉층의 정보로 부터 원하는 결과를 얻는 작업을 Decoding이라고 한다. <br>
> - 즉, Decoding이란 인코딩된 정보를 인간이 이해할 수 있는 표현으로 복원하는 작업이다.

__지금까지는 CBOW 모델을 뉴런의 관점에서 그렸다.<br>__
__이번에는 계층 관점에서 그려보도록 할 것__
![](img/layer_cbow1.png)

> - 편향을 사용하지 않는 완전연결계층의 처리는 __MatMul__계층의 순전파와 같다.
> - __MatMul__ 계층은 내부에서 행렬 곱을 계산한다.

__CBOW 모델의 추론 처리를 구현해보자. <br>__
__추론 처리란, 점수를 구하는 처리를 말한다.__

In [24]:
# ch03/cbow_predict.py
import numpy as np
from common.layers import MatMul

# 샘플 맥락 데이터
c0 = np.array([[1, 0, 0, 0, 0, 0, 0]])
c1 = np.array([[0, 0, 1, 0, 0, 0, 0]])

# 가중치 초기화
W_in = np.random.randn(7, 3)
W_out = np.random.randn(3, 7)

# 계층 생성
in_layer0 = MatMul(W_in)
in_layer1 = MatMul(W_in)
out_layer = MatMul(W_out)

#순전파
h0 = in_layer0.forward(c0)
h1 = in_layer1.forward(c1)
h = 0.5 * (h0 + h1)
s = out_layer.forward(h)

print(s)

[[-0.3317613   1.13446303 -0.84085377 -0.21437426 -2.17270732 -0.43708902
   0.11535379]]


여기까지가 CBOW 모델의 추론 과정이다. <br>
CBOW 모델은 활성화 함수를 사용하지 않는 간단한 구성의 신경망이다. <br><br>
그럼 계속해서 CBOW 모델의 학습에 대해 알아보자.

##### 3.2.2 CBOW 모델의 학습
지금까지 설명한 CBOW 모델은 출력층에서 각 단어의 점수를 출력했다.<br>
이 점수에 Softmax 함수를 적용하면 확률을 얻을 수 있다. <br>
이 확률은 맥락 : 전후 단어가 주어졌을 때, 그 중앙에 어떤 단어가 출현하는지를 나타낸다.

> 3.12 CBOW 모델의 구체적인 예 / 노드 값의 크기를 흑백의 진하기로 나타냈다.
![](img/cbow2.png)
> <br>
> CBOW 모델의 학습에서는 올바른 예측을 할 수 있도록 가중치를 조정하는 일을 한다.<br>
> 그 결과, 입력측/출력측 가중치에 단어의 출현 패턴을 파악한 벡터가 학습된다.<br>
> 이렇게 얻은 CBOW 모델과 skip-gram 모델로 얻을 수 있는 단어의 분산 표현은 단어의 의미 면에서나 문법 면에서 모두 우리의 직관에 부합한다.

신경망의 학습에 대해 생각해보자. <br>
우리가 다루는 모델은 다중 클래스 분류를 수행하는 신경망이다. <br>
따라서 이 신경망을 학습하려면, Softmax 함수와 교차 엔트로피 오차만 이용하면 된다. <br><br>

Softmax를 이용해 점수를 확률로 변환하고, <br>
그 확률로 정답 레이블로부터 교차 엔트로이 오차를 구한 후, <br>
그 값을 손실로 사용해 학습을 진행한다. <br>
> ![](img/cbow3.png)
> <br>추론 처리를 수행하는 CBOW 모델에 Softmax 계층과 Cross Entropy 계층을 추가한 것만으로도 손실을 얻을 수 있다. <br>
> 이상이 CBOW 모델의 손실을 구하는 계산 흐름이자, 이 신경망의 순방향 전파이다. <br>
> Softmax 계층과 Cross Entropy Error 계층은 Softmax with Loss 계층 하나로 구현할 수 있다.

____
![](img/cbow4.png)